<a href="https://colab.research.google.com/github/Dhanush-adk/machine_learning/blob/main/assignment_6/assignment_6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.7 MB/s eta 0:00:00
time: 252 µs (started: 2023-11-30 23:15:15 +00:00)


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

time: 926 µs (started: 2023-11-30 23:15:15 +00:00)


In [ ]:
torch.manual_seed(123)

time: 2.41 ms (started: 2023-11-30 23:15:15 +00:00)


In [ ]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

Files already downloaded and verified
time: 24 s (started: 2023-11-30 23:15:15 +00:00)


In [ ]:
mean

tensor([0.4914, 0.4822, 0.4465])

time: 3.92 ms (started: 2023-11-30 23:15:39 +00:00)


In [ ]:
std

tensor([0.2470, 0.2435, 0.2616])

time: 3.02 ms (started: 2023-11-30 23:15:39 +00:00)


In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

time: 618 µs (started: 2023-11-30 23:15:43 +00:00)


In [ ]:
device

device(type='cuda')

time: 2.83 ms (started: 2023-11-30 23:15:44 +00:00)


In [ ]:
# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
])

time: 662 µs (started: 2023-11-30 23:15:44 +00:00)


In [ ]:
cifar10 = datasets.CIFAR10(
    './data', train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


time: 705 ms (started: 2023-11-30 23:15:45 +00:00)


In [ ]:
cifar10_val = datasets.CIFAR10(
     './data', train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

time: 499 ms (started: 2023-11-30 23:15:47 +00:00)


In [ ]:
first_image, label = cifar10[0]
print(first_image.shape)

torch.Size([3, 32, 32])
time: 7.62 ms (started: 2023-11-30 23:15:48 +00:00)


In [ ]:
train_loader = DataLoader(cifar10, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(cifar10_val, batch_size=32, shuffle=False, num_workers=2)

time: 1.18 ms (started: 2023-11-30 23:19:05 +00:00)


In [ ]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.Tanh(),
    nn.Linear(512, 10)
).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

time: 44.6 ms (started: 2023-11-30 23:34:06 +00:00)


In [ ]:
def train_model(model, train_loader, test_loader, num_epochs=300, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Testing the model
        model.eval()
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = correct / total

        if epoch % 1 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Test Accuracy: {accuracy * 100:.2f}%')

    # Classification Report
    report = classification_report(all_labels, all_predicted)
    print("Classification Report:\n", report)


time: 1.59 ms (started: 2023-11-30 23:34:07 +00:00)


In [ ]:
train_model(model, train_loader, test_loader, num_epochs=300, lr=0.01)

Epoch 1/300, Loss: 1.7398397800484606, Test Accuracy: 41.89%
Epoch 2/300, Loss: 1.634406721675846, Test Accuracy: 43.21%
Epoch 3/300, Loss: 1.5660125428258953, Test Accuracy: 45.81%
Epoch 4/300, Loss: 1.5045548411645115, Test Accuracy: 45.98%
Epoch 5/300, Loss: 1.4462770357089247, Test Accuracy: 46.90%
Epoch 6/300, Loss: 1.393377955266473, Test Accuracy: 47.19%
Epoch 7/300, Loss: 1.33971855736511, Test Accuracy: 46.97%
Epoch 8/300, Loss: 1.286995179517408, Test Accuracy: 47.02%
Epoch 9/300, Loss: 1.2365475362184637, Test Accuracy: 47.75%
Epoch 10/300, Loss: 1.1836568696027525, Test Accuracy: 48.50%
Epoch 11/300, Loss: 1.1335558970616708, Test Accuracy: 48.34%
Epoch 12/300, Loss: 1.0809114643265938, Test Accuracy: 48.87%
Epoch 13/300, Loss: 1.029821600467062, Test Accuracy: 47.75%
Epoch 14/300, Loss: 0.9790819698774273, Test Accuracy: 48.79%
Epoch 15/300, Loss: 0.9285842062224964, Test Accuracy: 47.34%
Epoch 16/300, Loss: 0.8829675394986886, Test Accuracy: 46.96%
Epoch 17/300, Loss: 0.8

In [ ]:
model2 = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32 * 32 * 3, 512),
    nn.Tanh(),
    nn.Linear(512, 256),
    nn.Tanh(),
    nn.Linear(256, 128),
    nn.Tanh(),
    nn.Linear(128, 10)
).to(device)

time: 31.5 ms (started: 2023-12-01 01:09:11 +00:00)


In [ ]:
train_model(model2, train_loader, test_loader, num_epochs=300, lr=0.01)

Epoch 1/300, Loss: 1.8508661996075075, Test Accuracy: 39.19%
Epoch 2/300, Loss: 1.6860174748352987, Test Accuracy: 42.09%
Epoch 3/300, Loss: 1.613621175937445, Test Accuracy: 43.42%
Epoch 4/300, Loss: 1.5521392714725575, Test Accuracy: 45.22%
Epoch 5/300, Loss: 1.4946926155306937, Test Accuracy: 46.36%
Epoch 6/300, Loss: 1.4416150330581958, Test Accuracy: 47.50%
Epoch 7/300, Loss: 1.3840028547889822, Test Accuracy: 47.84%
Epoch 8/300, Loss: 1.3247861227238704, Test Accuracy: 48.54%
Epoch 9/300, Loss: 1.2697526247014774, Test Accuracy: 48.28%
Epoch 10/300, Loss: 1.2086391765493196, Test Accuracy: 48.70%
Epoch 11/300, Loss: 1.1492945738732625, Test Accuracy: 47.45%
Epoch 12/300, Loss: 1.0891687711384994, Test Accuracy: 46.37%
Epoch 13/300, Loss: 1.0271452304192712, Test Accuracy: 48.22%
Epoch 14/300, Loss: 0.9693194204465899, Test Accuracy: 47.59%
Epoch 15/300, Loss: 0.9086546413385936, Test Accuracy: 47.31%
Epoch 16/300, Loss: 0.8466312264450376, Test Accuracy: 47.46%
Epoch 17/300, Loss